In [1]:
# Importar Bibliotecas
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.initializers import Zeros
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Preparación de datos

# Cargar el conjunto de datos MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()


# Imprimir las dimensiones de los conjuntos de entrenamiento y prueba
print("Datos de entrenamiento (x_train): ", x_train.shape)
print("Etiquetas de entrenamiento (y_train): ", y_train.shape)
print("Datos de prueba (x_test): ", x_test.shape)
print("Etiquetas de prueba (y_test): ", y_test.shape)

# Mostrar las primeras 5 imágenes del conjunto de entrenamiento con sus etiquetas
for i in range(5):
    plt.subplot(1, 5, i+1)
    plt.imshow(x_train[i], cmap='gray')
    plt.title("Etiqueta: " + str(y_train[i]))
    plt.axis('off')
plt.show()


# Mostrar el número original de imágenes por clase
print("Número original de imágenes por clase en el conjunto de entrenamiento:")
unique, counts = np.unique(y_train, return_counts=True)
print(dict(zip(unique, counts)))

# Normalizar los datos: Los valores de píxeles en imágenes en escala de grises están en el rango 0-255 (8 bits por píxel).
# Al dividir por 255.0, transformamos estos valores a un rango de 0-1 para facilitar el entrenamiento de la red neuronal.
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convertir las etiquetas a binario: 1 para 'cinco' y 0 para 'no cinco'
y_train_bin = (y_train == 5).astype(np.float32)
y_test_bin = (y_test == 5).astype(np.float32)

# Balancear el conjunto de entrenamiento para tener aproximadamente el mismo número de cincos y no cincos
x_cincos = x_train[y_train_bin == 1]
y_cincos = y_train_bin[y_train_bin == 1]
x_no_cincos = x_train[y_train_bin == 0][:len(x_cincos)]
y_no_cincos = y_train_bin[y_train_bin == 0][:len(y_cincos)]

# * Concatenación de arrays con cincos y otros números
x_train_balanced = np.concatenate((x_cincos, x_no_cincos), axis=0)
y_train_balanced = np.concatenate((y_cincos, y_no_cincos), axis=0)

# Verificación del balanceo
print("\nDespués del balanceo:")
print("Número de 'cincos':", np.sum(y_train_balanced == 1))
print("Número de 'no cincos':", np.sum(y_train_balanced == 0))

# Mezclar los datos balanceados
x_train_balanced, y_train_balanced = resample(x_train_balanced, y_train_balanced)

# Mostrar el número despues del balanceo de imágenes por clase
print("Número original de imágenes por clase en el conjunto de entrenamiento:")
unique, counts = np.unique(y_train_balanced, return_counts=True)
print("Recuerda que 1.0 es un 5 y 0.0 es cualquier otro número")
print(dict(zip(unique, counts)))

# Aplanar las imágenes para la entrada a la red neuronal
x_train_balanced = x_train_balanced.reshape((x_train_balanced.shape[0], -1))
x_test = x_test.reshape((x_test.shape[0], -1))

# Mostrar dimensiones finales de los conjuntos de datos
print("\nDimensiones finales:")
print("x_train_balanced:", x_train_balanced.shape)
print("y_train_balanced:", y_train_balanced.shape)
print("x_test:", x_test.shape)
print("y_test_bin:", y_test_bin.shape)

# Dividir los datos de entrenamiento balanceados en conjuntos de entrenamiento y validación
x_train_balanced, x_val_balanced, y_train_balanced, y_val_balanced = train_test_split(
    x_train_balanced, y_train_balanced, test_size=0.2, random_state=42)

# Mostrar las dimensiones de los nuevos conjuntos de entrenamiento y validación
print("Dimensiones de entrenamiento balanceado: ", x_train_balanced.shape, y_train_balanced.shape)
print("Dimensiones de validación balanceado: ", x_val_balanced.shape, y_val_balanced.shape)

In [ ]:
# Construcción de la Red Neuronal

model = Sequential()

# Añadir la primera capa (capa oculta) con 2 neuronas y función de activación 'relu'
# 'input_shape' define la forma de los datos de entrada (784 características por imagen)
model.add(Dense(2, activation='relu', input_shape=(784,)))
    
# Añadir la capa de salida con 1 neurona y función de activación 'sigmoid'
# 'sigmoid' es adecuada para la clasificación binaria
model.add(Dense(1, activation='sigmoid'))

# Compilar el modelo
# 'binary_crossentropy' es la función de pérdida adecuada para clasificación binaria
# 'adam' es un optimizador efectivo para muchos problemas de aprendizaje automático
# 'accuracy' como métrica para monitorear durante el entrenamiento
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Compilar el modelo
# 'binary_crossentropy' es la función de pérdida adecuada para clasificación binaria
# 'adam' es un optimizador efectivo para muchos problemas de aprendizaje automático
# 'accuracy' como métrica para monitorear durante el entrenamiento
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Imprimir un resumen del modelo para verificar la estructura
model.summary()

In [ ]:
# Configurar Early Stopping
# Monitorear la 'accuracy' en el conjunto de validación y detener el entrenamiento cuando deje de mejorar
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Entrenar el modelo
# Elegir un 'batch size' y 'validation split' adecuados
history = model.fit(x_train_balanced, y_train_balanced, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Visualizar el proceso de entrenamiento
# Graficar el 'accuracy' y 'loss' para el entrenamiento y la validación
plt.figure(figsize=(14, 5))

# Graficar Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

# Graficar Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()

plt.show()


In [ ]:
# Realizar predicciones en el conjunto de prueba
predicciones = model.predict(x_test)
# Las predicciones estarán en formato de probabilidad, conviértelas a clases binarias (0 o 1)
predicciones_clases = (predicciones > 0.5).astype("int32")

# Identificar los índices de los errores de clasificación
errores_indices = np.where(predicciones_clases.flatten() != y_test_bin)[0]

# Seleccionar al azar o los primeros 80 errores para mostrar
errores_muestra_indices = np.random.choice(errores_indices, 80, replace=False)

# Mostrar las imágenes mal clasificadas
plt.figure(figsize=(20, 20))
for i, indice_error in enumerate(errores_muestra_indices[:80], start=1):
    plt.subplot(10, 8, i)
    plt.imshow(x_test[indice_error].reshape(28, 28), cmap='gray')
    plt.title(f"Pred: {predicciones_clases[indice_error][0]}, Real: {y_test_bin[indice_error]}")
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Ya que las predicciones están en formato de probabilidad, usamos un umbral de 0.5 para clasificar como 0 o 1
y_pred = (model.predict(x_test) > 0.5).astype("int32")

# Calcular las métricas
accuracy = accuracy_score(y_test_bin, y_pred)
precision = precision_score(y_test_bin, y_pred)
recall = recall_score(y_test_bin, y_pred)
f1 = f1_score(y_test_bin, y_pred)

# Mostrar las métricas
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

In [ ]:
# Calcular la curva ROC y el AUC
fpr, tpr, thresholds = roc_curve(y_test_bin, model.predict(x_test))
roc_auc = auc(fpr, tpr)

# Graficar la curva ROC
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()
